In [3]:
import os
os.system('pip install surprise')

0

In [5]:
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')

trainset = data.build_full_trainset()
for i, (uid, iid, rating) in enumerate(trainset.all_ratings()):
    if i < 5:
        print(f"User: {uid}, Item: {iid}, Rating: {rating}")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\faika/.surprise_data/ml-100k
User: 0, Item: 0, Rating: 3.0
User: 0, Item: 528, Rating: 4.0
User: 0, Item: 377, Rating: 4.0
User: 0, Item: 522, Rating: 3.0
User: 0, Item: 431, Rating: 5.0


In [7]:
from surprise import SVD, KNNBaseline
from surprise.model_selection import cross_validate

algo_svd = SVD()
algo_knn = KNNBaseline()

print("Cross-validation for SVD:")
cross_validate(algo_svd, data, measures=['MAE'], cv=5, verbose=True)

print("\nCross-validation for KNNBaseline:")
cross_validate(algo_knn, data, measures=['MAE'], cv=5, verbose=True)

Cross-validation for SVD:
Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7357  0.7319  0.7389  0.7346  0.7478  0.7378  0.0055  
Fit time          0.37    0.35    0.35    0.36    0.40    0.37    0.02    
Test time         0.05    0.10    0.05    0.05    0.10    0.07    0.02    

Cross-validation for KNNBaseline:
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBaseline on 5 split(s).

               

{'test_mae': array([0.7299564 , 0.7302508 , 0.73018245, 0.73764575, 0.73783459]),
 'fit_time': (0.22958159446716309,
  0.2375657558441162,
  0.2433483600616455,
  0.23595333099365234,
  0.23600292205810547),
 'test_time': (1.2819154262542725,
  1.33559250831604,
  1.257181167602539,
  1.2909514904022217,
  1.2350094318389893)}

In [9]:
from surprise.model_selection import GridSearchCV

param_grid_svd = {'n_factors': [50, 100], 'lr_all': [0.005, 0.01], 'reg_all': [0.02, 0.05]}
param_grid_knn = {'k': [20, 40], 'min_k': [1, 5]}

gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['mae'], cv=3)
gs_svd.fit(data)

gs_knn = GridSearchCV(KNNBaseline, param_grid_knn, measures=['mae'], cv=3)
gs_knn.fit(data)

print(f"Best SVD params: {gs_svd.best_params['mae']}")
print(f"Best KNN params: {gs_knn.best_params['mae']}")

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [11]:
if gs_svd.best_score['mae'] < gs_knn.best_score['mae']:
    best_algo = gs_svd.best_estimator['mae']
    print(f"Best algorithm is SVD with MAE: {gs_svd.best_score['mae']}")
else:
    best_algo = gs_knn.best_estimator['mae']
    print(f"Best algorithm is KNNBaseline with MAE: {gs_knn.best_score['mae']}")

Best algorithm is SVD with MAE: 0.7345788038471651


In [17]:
trainset = data.build_full_trainset()
best_algo.fit(trainset)

user_id = str(196)
items = trainset.all_items()

recommendations = []
for item_id in items:
    predictions = best_algo.predict(user_id, trainset.to_raw_iid(item_id))
    recommendations.append((item_id, predictions.est))

recommendations.sort(key=lambda x: x[1], reverse=True)
top_10 = recommendations[:10]

print("Top 10 recommended items for user 196:")
for item in top_10:
    print(item)

Top 10 recommended items for user 196:
(239, 4.6656409319191265)
(200, 4.656459741377527)
(189, 4.617439183434073)
(569, 4.589609397053106)
(359, 4.546539598328852)
(31, 4.516932124336967)
(78, 4.512721047925106)
(719, 4.4939508023522015)
(254, 4.48975458339188)
(361, 4.478988846995685)


Завдання 2

In [37]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2)

algo = SVD()
algo.fit(trainset)

predictions = algo.test(testset)

def get_top_n_recommendations(algo, user_id, trainset, n=10):
    all_items = trainset.all_items()
    rated_items = set([iid for (iid, _) in trainset.ur[trainset.to_inner_uid(user_id)]])
    
    recommendations = []
    for iid in all_items:
        if iid not in rated_items:
            raw_iid = trainset.to_raw_iid(iid)
            est_rating = algo.predict(user_id, raw_iid).est
            recommendations.append((raw_iid, est_rating))

    recommendations.sort(key=lambda x: x[1], reverse=True)

    return recommendations[:n]

user_id = str(196)
top_10_recommendations = get_top_n_recommendations(algo, user_id, trainset)

print(f"Top 10 recommended movies for user {user_id}:")
for movie_id, rating in top_10_recommendations:
    print(f"Movie ID: {movie_id}, Predicted Rating: {rating:.2f}")

Top 10 recommended movies for user 196:
Movie ID: 318, Predicted Rating: 4.81
Movie ID: 114, Predicted Rating: 4.76
Movie ID: 187, Predicted Rating: 4.73
Movie ID: 483, Predicted Rating: 4.62
Movie ID: 251, Predicted Rating: 4.58
Movie ID: 169, Predicted Rating: 4.57
Movie ID: 357, Predicted Rating: 4.55
Movie ID: 190, Predicted Rating: 4.55
Movie ID: 127, Predicted Rating: 4.54
Movie ID: 657, Predicted Rating: 4.52
